# Working with multi-variable assets

In addition to catalogs of data assets (files) in time-series (single-variable)
format, intake-esm supports catalogs with data assets in time-slice (history)
format and/or files with multiple variables. For intake-esm to properly work
with multi-variable assets,

- the `variable_column` of the catalog must contain iterables (list, tuple, set)
  of values.
- the user must specifiy a dictionary of functions for converting values in
  certain columns into iterables. This is done via the `csv_kwargs` argument.

In the example below, we are are going to use the following catalog to
demonstrate how to work with multi-variable assets:


In [ ]:
# Look at the catalog on disk
!cat multi-variable-catalog.csv

As you can see, the variable column contains a list of varibles, and this list
was serialized as a string:
`"['SHF', 'REGION_MASK', 'ANGLE', 'DXU', 'KMT', 'NO2', 'O2']"`.


## Loading a catalog

To load a catalog with multiple variable files, we must pass additional
information to `open_esm_datastore` via the `csv_kwargs` argument. We are going
to specify a dictionary of functions for converting values in `variable` column
into iterables. We use the `literal_eval` function from the standard `ast`
module:


In [ ]:
import ast

import intake

In [ ]:
col = intake.open_esm_datastore(
    "multi-variable-collection.json",
    csv_kwargs={"converters": {"variable": ast.literal_eval}},
)
col

In [ ]:
col.df.head()

The in-memory representation of the catalog contains `variable` with tuple of
values. To confirm that intake-esm has registered this catalog with multiple
variable assets, we can the `._multiple_variable_assets` property:


In [ ]:
col._multiple_variable_assets

## Searching

The search functionatilty works in the same way:


In [ ]:
col_subset = col.search(variable=["O2", "SiO3"])
col_subset.df

## Loading assets into xarray datasets

Loading data assets into xarray datasets works in the same way too:


In [ ]:
col_subset.to_dataset_dict(cdf_kwargs={})

In [ ]:
import intake_esm  # just to display version information

intake_esm.show_versions()